In [38]:
import math
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import FixedTicker,Legend,Plot,GraphRenderer, Oval, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool,BoxZoomTool, Rect, WheelZoomTool
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes,from_networkx
from bokeh.palettes import Spectral4, Spectral8, Blues8, Greens8, magma,OrRd,Oranges, Viridis256, Oranges8
from bokeh.plotting import ColumnDataSource
from bokeh.models import ColumnDataSource, LabelSet, ColumnData,StaticLayoutProvider,GraphRenderer
from bokeh.layouts import gridplot
import numpy as np
from networkx.algorithms import approximation as approx
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np
import csv

In [2]:
#Importing Nodes as Pandas DF
nodes = pd.read_csv("sheep_data/sheep_age.csv")
edges = pd.read_csv("sheep_data/sheep_edges.csv")

In [3]:
#Importing Edges as Tuples 
with open('sheep_data/sheep_edges.csv') as f:
    sheep_edges=[tuple(line) for line in csv.reader(f)]

In [4]:
sheep_edges.pop(0)

('source', 'target', 'weight')

In [5]:
#Creating the Network
X = nx.DiGraph()

In [6]:
#D.add_weighted_edges_from(sheep_edges)
X.add_weighted_edges_from(sheep_edges)

In [7]:
#Loading as graphml
#new = nx.read_graphml('sheep_data/sheep_ml.graphml')
#print("Number of Nodes: ", new.number_of_nodes())
#print("Number of Edges: ", new.number_of_edges())

In [8]:
#Creating Features
#1 - How many times that particular sheep has dominated another sheep
src_dom = edges.groupby(['source']).sum()
#2 - How many times that particular sheep has been dominated by another sheep
tgt_dom = edges.groupby(['target']).sum()

In [9]:
src_dom = src_dom.reset_index()
tgt_dom = tgt_dom.reset_index()

In [10]:
df2 = pd.DataFrame([[1, 0, 0]],columns=['target','source','weight'])
tgt_dom = tgt_dom.append(df2)
tgt_dom = tgt_dom.sort_values('target')
tgt_dom = tgt_dom.drop(columns=['source'])
tgt_dom.rename(columns={'target': 'id', 'weight':'DominatedBy'}, inplace=True)

In [11]:
df2 = pd.DataFrame([[6, 0, 0]],columns=['source', 'target','weight'])
src_dom = src_dom.append(df2)
src_dom = src_dom.sort_values('source')
src_dom = src_dom.drop(columns=['target'])
src_dom.rename(columns={'source': 'id', 'weight':'DominatedOthers'}, inplace=True)                                                                  

In [12]:
#Merge Original Nodes DF to add Dominated
nodes = pd.merge(nodes, tgt_dom, how='inner', on='id')
nodes = pd.merge(nodes, src_dom, how='inner', on='id')

In [13]:
#1 - How many different sheep it dominated
src_diff = edges.groupby(['source']).sum()
#2 - How many different sheep it was dominated by
tgt_diff = edges.groupby(['target']).sum()

In [14]:
edges_work = edges.copy()
edges_work = edges_work.drop(columns=['weight'])

In [15]:
#del src
src = edges_work.groupby(['source']).count()
src = src.reset_index()
df2 = pd.DataFrame([[6, 0]],columns=['source', 'target'])
src = src.append(df2)
src = src.sort_values('source')
src.rename(columns={'source': 'id', 'target':'TargetsPerSource'}, inplace=True)

In [16]:
edges_work = edges.copy()
edges_work = edges_work.drop(columns=['weight'])

In [17]:
#del tgt
tgt = edges_work.groupby('target').count()
tgt = tgt.reset_index()
df2 = pd.DataFrame([[1, 0]],columns=['target', 'source'])
tgt = tgt.append(df2)
tgt = tgt.sort_values('target')
tgt.rename(columns={'target': 'id', 'source':'SourcesPerTarget'}, inplace=True)

In [18]:
#Adding Plus One to make division possible 
nodes['DominatedOthers'] = nodes['DominatedOthers']+1

In [19]:
nodes['DominatedBy'] = nodes['DominatedBy']+1

In [20]:
nodes = pd.merge(nodes, src, how='inner', on='id')
nodes = pd.merge(nodes, tgt, how='inner', on='id')

In [21]:
nodes['TargetsPerSource'] = nodes['TargetsPerSource']+1

In [22]:
nodes['SourcesPerTarget'] = nodes['SourcesPerTarget']+1

In [23]:
nodes['DomBy/DomOthers'] = nodes['DominatedBy'].divide(nodes['DominatedOthers'])
nodes['Tgts/Srcs'] = nodes['TargetsPerSource'].divide(nodes['SourcesPerTarget'])

In [24]:
nodes['DomBy/DomOthers'] = nodes['DomBy/DomOthers']+.1

In [25]:
#Load the node attributes
for j in nodes.columns:
    for i,k in zip(sorted(X.nodes()),pd.Series(nodes[j])):
        X.node[i][j] = k

In [26]:
#Create Edge Attributes:
color_series = []
for i in edges['weight']:
    if i>7:
        color_series.append(Spectral8[7])
    else:
        color_series.append(Spectral8[i])

In [27]:
edges['weight_color'] = color_series

In [28]:
wgt_col = {'weight_color': edges['weight_color'].to_dict()}

In [29]:
nodes['DominatedOthersPCGT'] = nodes['DominatedOthers']/nodes['DominatedOthers'].sum()

In [30]:
nodes['DominatedByPCGT'] = nodes['DominatedBy']/nodes['DominatedBy'].sum()

In [31]:
nodes['TotalInteractions'] = nodes['DominatedOthers']+nodes['DominatedBy']
nodes['WonPCGT'] = nodes['DominatedOthers']/nodes['TotalInteractions']
nodes['LostPCGT'] = nodes['DominatedBy']/nodes['TotalInteractions']

In [32]:
#NetworkX Algorithms

In [33]:
nodes.head(2)

,id,age,DominatedBy,DominatedOthers,TargetsPerSource,SourcesPerTarget,DomBy/DomOthers,Tgts/Srcs,DominatedOthersPCGT,DominatedByPCGT,TotalInteractions,WonPCGT,LostPCGT
0,1,8,1,6,6,1,0.266667,6.0,0.008746,0.001458,7,0.857143,0.142857
1,2,8,10,36,15,6,0.377778,2.5,0.052478,0.014577,46,0.782609,0.217391


In [ ]:
print("Pairwise or local node connectivity between two distinct and nonadjacent nodes")
print("Minimum number of nodes that must be removed (minimum separating cutset) to disconnect Nodes 1 and 2: ")
print(approx.node_connectivity(X,'1','2'))

In [ ]:
print("A clique in an undirected graph G = (V, E) is a subset of the vertex set C subset e q V such that") 
print("for every two vertices in C there exists an edge connecting the two")
print("Max Clique: ", approx.max_clique(X))
mx_clique = approx.max_clique(X)


In [ ]:
print("A maximum independent set is a largest independent set for a given graph G and its size is denoted 𝛼(G).")
print("The maximum independent set of New is: ", approx.maximum_independent_set(X))
mx_indep = approx.maximum_independent_set(X)

In [ ]:
print("Given a graph G = (V,E), a matching M in G is a set of pairwise non-adjacent edges; that is, no two edges share a common vertex")
print("The Matching of new is: ", approx.min_maximal_matching(X))

In [ ]:
print("The degree centrality for a node v is the fraction of nodes it is connected to.")
print("The degree centrality for each node is ", nx.degree_centrality(X))

In [ ]:
print("The in-degree centrality for a node v is the fraction of nodes its incoming edges are connected to")
print("The in-degree centrality for each node is ", nx.in_degree_centrality(X))

In [ ]:
print("The out-degree centrality for a node v is the fraction of nodes its outgoing edges are connected to.")
print("The out-degree centrality for each node is ", nx.out_degree_centrality(X))

In [ ]:
print("Betweennes centrality for each node in new: ", nx.betweenness_centrality(X))

In [ ]:
print("Betweennes centrality for each edge in new: ", nx.edge_betweenness_centrality(X))

In [ ]:
print("The local reaching centrality of a node in a directed graph is the proportion of other nodes reachable from that node")
print("The local reaching centrality for node 4 is: ", nx.local_reaching_centrality(X, "4"))

In [ ]:
nx.greedy_color(X, strategy="largest_first", interchange=False)

In [ ]:
# or DiGraph, MultiGraph, MultiDiGraph, etc
mx_cliq = list(mx_clique)
H = nx.DiGraph(X.subgraph(mx_cliq))
type(H.edges())

In [ ]:
H_edges_lst = []
for i in H.edges():
    H_edges_lst.append(i)

H_edges = pd.DataFrame(H_edges_lst, columns=['source', 'target'])
H_edges1 = pd.merge(edges, H_edges, how='right', left_on=['source','target'],right_on=['source','target'])
H_edges1=H_edges1.drop(columns=['weight_color'])

In [ ]:
edges1 = edges.copy()
edges1 = edges

In [ ]:
nds_dom_pt_lst = []
for i in nodes['DominatedOthersPCGT'].tolist():
    nds_dom_pt_lst.append(str(i))

In [ ]:
edge_weight_lst = []
for edge in edges['weight']:
    edge_weight_lst.append(str(edge))
#pass dictionary and e - example is at the main page (latest)

In [ ]:
nodes_list = []
for i in X.nodes:
    nodes_list.append(i)
print(nodes_list)

In [ ]:
edge_sorted = edges.sort_values(by=['weight'])

In [ ]:
#output_notebook()

plot = figure(title="Networkx Integration Demonstration", x_range=(-2,2), y_range=(-2,2),
              tools="", toolbar_location=None)

graph = from_networkx(X, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)
show(plot)

In [138]:
#First Plot
N1 = len(X.nodes)
node_indices = list(range(N1))
nd_idx = [i+1 for i in node_indices]

p = figure(title="Graph Spectral Layout", x_range=(-1.1,1.1), y_range=(-1.1,1.1), toolbar_location="right")

p.title.text_color = "black"
p.title.text_font = "verdana"
p.title.text_font_style = "italic"
p.title.text_font_style = "bold"
p.background_fill_alpha = 0.07
p.title.align = "center"
p.title.text_font_size = '29pt'

p.add_tools(HoverTool(tooltips=[("Source", "@edges_src"),
                                ("Target", "@edges_tgt"),("Weight","@width")]), 
            TapTool(), BoxSelectTool(), BoxZoomTool(),WheelZoomTool())

graph_renderer = from_networkx(X,nx.spectral_layout, center=(0,0))

pos = nx.spectral_layout(X)
x,y=zip(*pos.values())

src = ColumnDataSource({'x':x,'y':y,'nodeid':nd_idx})
labels = LabelSet(x='x', y='y', text='nodeid', source=src)

graph_renderer.node_renderer.glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color"
                                            
                                            )

graph_renderer.node_renderer.selection_glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph_renderer.node_renderer.hover_glyph = Circle(size="size_dim", fill_alpha="dom_index", 
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph_renderer.node_renderer.data_source.data = dict(index=nd_idx, 
                                            size_dim = nodes['DominatedOthersPCGT']*600,
                                            dom_index = 30/nodes['DomBy/DomOthers'],
                                            dom_by = nodes['DominatedBy']/7,
                                            fi_color = nodes['DomBy/DomOthers_color'],
                                            age = nodes['age']/60
                                                    )

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#bdbdbd", line_alpha="width_alpha",
                                               line_dash="dotted", line_width="width")
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color="edge_col", line_width="width")
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color="edge_col", line_width="width")
graph_renderer.edge_renderer.data_source.data = dict(start=edges['source'],
                                                     end=edges['target'],
                                                     width=edges['weight'],
                                                     edge_col=edges['weight_color'],
                                                     width_alpha=edges['weight']/2,
                                                     edges_src=edges['source'],
                                                     edges_tgt=edges['target']
                                                    )

legend = Legend(items=[("Node ID",[])], location="top_right")

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

p.renderers.append(labels)
p.renderers.append(graph_renderer)
p.add_layout(legend)
show(p)


E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='1b09c1a0-d23d-4c7c-942d-56e4ef42a56a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='8732117f-1202-479d-8f65-d3b7e51bf656', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: age, fi_color2, tgtdivsrc, tgtdivsrcstgt [renderer: GlyphRenderer(id='366d8f7d-9b09-42a4-8b72-b9c538ec0907', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='4a76762b-fa57-4933-a52c-68811dbfa04e', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='8504fcdf-6c25-46db-9570-2072e93e711d', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='ce7743b0-6548-4735-8eb8-8

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='bfc12413-3256-497d-9f91-9aab57015001', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='ca4484d8-e08d-44a3-99ed-44c3d0cd6e5d', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: age, fi_color2, fi_color3, tgtdivsrcstgt [renderer: GlyphRenderer(id='bc4a3310-59de-4d44-8eb2-085a2f36456a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='dcb1744b-6859-4984-a759-22dc100bb6e7', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='0c974d06-969e-4f88-9503-b622b59dec92', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='656c8002-59c7-42f4-a8e6-b76c148d5807', ...)]

In [136]:
#First Plot
N1 = len(X.nodes)
node_indices = list(range(N1))
plot_options = dict(width=400, plot_height=400)
nds_dom_pt=nodes['DominatedOthersPCGT'].tolist()
nd_idx = [i+1 for i in node_indices]
#tgtdivsrcstgt-tgtdivsrc
node_src = dict(index=nd_idx, dom_oth_pcgt = nds_dom_pt,
                                    size_dim = nodes['DominatedOthersPCGT']*300,
                                    dom_index = 30/nodes['DomBy/DomOthers'],
                                    dom_by = nodes['DominatedBy']/7,
                                    fi_color = nodes['DomBy/DomOthers_color'],
                                    fi_color2 = nodes['Tgts/Srcs_color'],
                                    fi_color3 = nodes['WonColor'],
                                    age = nodes['age']/60,
                                    tgtdivsrcstgt=nodes['Tgts/Srcs']*30#,
                                    #tgtdivsrc = 1              
               )

edge_src = dict(start=edges['source'],end=edges['target'],
                                     width=edges['weight'],
                                     edge_col=edges['weight_color'],
                                     width_alpha=edges['weight']/2,
                                     edges_src=edges['source'],
                                     edges_tgt=edges['target'])

s1 = figure(title="Graph Layout Demonstration", x_range=(-1.1,1.1)
            , y_range=(-1.1,1.1),tools='pan,wheel_zoom,tap,xbox_select', toolbar_location="right", **plot_options)

s1.add_tools(HoverTool(tooltips=[("Source", "@edges_src"),
                                ("Target", "@edges_tgt"),("Weight","@width")]), 
            TapTool(), BoxSelectTool(), BoxZoomTool(),WheelZoomTool())

graph1 = from_networkx(X,nx.shell_layout, center=(0,0))

pos = nx.shell_layout(X)
x,y=zip(*pos.values())

src = ColumnDataSource({'x':x,'y':y,'nodeid':nd_idx})
labels = LabelSet(x='x', y='y', text='nodeid', source=src)
s1.title.text_color = "blue"
s1.title.text_font = "verdana"
s1.title.text_font_style = "italic"

graph1.node_renderer.glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color"
                                            
                                            )
graph1.node_renderer.selection_glyph = Circle(size="size_dim", fill_alpha="dom_index",
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph1.node_renderer.hover_glyph = Circle(size="size_dim", fill_alpha="dom_index", 
                                            line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color")

graph1.node_renderer.data_source.data = node_src

graph1.edge_renderer.glyph = MultiLine(line_color="#bdbdbd", line_alpha="width_alpha",
                                               line_dash="dotted", line_width="width")

graph1.edge_renderer.selection_glyph = MultiLine(line_color="edge_col", line_width="width")
graph1.edge_renderer.hover_glyph = MultiLine(line_color="edge_col", line_width="width")
graph1.edge_renderer.data_source.data = edge_src

legend1 = Legend(items=[("Node ID",[])], location="top_right")


graph1.selection_policy = NodesAndLinkedEdges()
graph1.inspection_policy = EdgesAndLinkedNodes()

s1.renderers.append(graph1)

s1.renderers.append(labels)
s1.renderers.append(graph1)
s1.add_layout(legend1)

##Second Plot
N2 = len(X.nodes)
node_indices2 = list(range(N2))

s2 = figure(x_range=s1.x_range, y_range=s1.y_range,
               title="Second Graph",tools='pan,wheel_zoom,tap,xbox_select', 
                toolbar_location="right", **plot_options)

s2.add_tools(HoverTool(tooltips=[("Source", "@edges_src"),
                                ("Target", "@edges_tgt"),("Weight","@width")]), 
            TapTool(), BoxSelectTool(), BoxZoomTool(),WheelZoomTool())

graph2 = from_networkx(X,nx.shell_layout, center=(0,0))

s2.title.text_color = "blue"
s2.title.text_font = "verdana"
s2.title.text_font_style = "italic"                                    

graph2.node_renderer.glyph = Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "fi_color3",
                                            line_dash = "solid",
                                            line_width = 5,#"tgtdivsrc",
                                            fill_color="fi_color2"
                                 )
graph2.node_renderer.selection_glyph = Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "fi_color3",
                                            line_dash = "solid",
                                            line_width = 5,#"tgtdivsrc",
                                            fill_color="fi_color2"
                                            
                                            )

graph2.node_renderer.hover_glyph = Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "fi_color3",
                                            line_dash = "solid",
                                            line_width = 5,#"tgtdivsrc",
                                            fill_color="fi_color2"
                                            
                                            )

graph2.node_renderer.data_source.data = node_src

graph2.edge_renderer.glyph = MultiLine(line_color="#bdbdbd", line_alpha="width_alpha",
                                               line_dash="dotted", line_width="width")

graph2.edge_renderer.selection_glyph = MultiLine(line_color="edge_col", line_width="width")
graph2.edge_renderer.hover_glyph = MultiLine(line_color="edge_col", line_width="width")
graph2.edge_renderer.data_source.data = edge_src


legend2 = Legend(items=[("Node ID",[])], location="top_right")

graph2.selection_policy = NodesAndLinkedEdges()
graph2.inspection_policy = EdgesAndLinkedNodes()

s2.renderers.append(graph2)

s2.renderers.append(labels)
s2.renderers.append(graph2)
s2.add_layout(legend2)

p = gridplot([[s1, s2]])

show(p)


E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='1b09c1a0-d23d-4c7c-942d-56e4ef42a56a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='8732117f-1202-479d-8f65-d3b7e51bf656', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: age, fi_color2, tgtdivsrc, tgtdivsrcstgt [renderer: GlyphRenderer(id='366d8f7d-9b09-42a4-8b72-b9c538ec0907', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='4a76762b-fa57-4933-a52c-68811dbfa04e', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='8504fcdf-6c25-46db-9570-2072e93e711d', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='ce7743b0-6548-4735-8eb8-8

E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='bfc12413-3256-497d-9f91-9aab57015001', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='ca4484d8-e08d-44a3-99ed-44c3d0cd6e5d', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: age, fi_color2, fi_color3, tgtdivsrcstgt [renderer: GlyphRenderer(id='bc4a3310-59de-4d44-8eb2-085a2f36456a', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: dom_by, dom_index, fi_color, size_dim [renderer: GlyphRenderer(id='dcb1744b-6859-4984-a759-22dc100bb6e7', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='0c974d06-969e-4f88-9503-b622b59dec92', ...)]
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: width, width_alpha [renderer: GlyphRenderer(id='656c8002-59c7-42f4-a8e6-b76c148d5807', ...)]

In [ ]:
nodes.head(2)

In [ ]:
#fillalpha: nodes['TargetsPerSource']/nodes['DominatedOthers'])*10,
#linewidth: nodes['Tgts/Srcs']

In [ ]:
type(Spectral8)

In [49]:
from bokeh.models import LinearColorMapper, ColorBar, Ticker, LogTicker, BasicTicker
from bokeh.palettes import YlOrBr8,Oranges8

source = ColumnDataSource(dict(nodes))
color_mapper = LinearColorMapper(palette=Oranges8, low=nodes.WonPCGT.max(), high=nodes.WonPCGT.min())

tls="crosshair,pan,wheel_zoom,box_zoom,reset,hover,previewsave"

p = figure(x_axis_label='TargetsPerSource', y_axis_label='SourcesPerTarget', tools=tls, toolbar_location='above')
p.circle(x='TargetsPerSource', y='SourcesPerTarget', color={'field': 'WonPCGT', 'transform': color_mapper}, 
         size=20, alpha=0.6, source=source)
hover =p.select(dict(type=HoverTool))
hover.tooltips = dict([
    ("Node ID", "@id"),
])

color_bar = ColorBar(color_mapper=color_mapper, 
                     label_standoff=12, 
                     location=(0,0), 
                     orientation='vertical', 
                     title='WonPCGT',
                    ticker=BasicTicker())

p.add_layout(color_bar, 'right')

show(p)

In [ ]:
dict(index=node_indices, dom_oth_pcgt = nds_dom_pt,  size_dim = nodes['DominatedOthersPCGT']*400,
                                    dom_index = 30/nodes['DomBy/DomOthers'],
                                    dom_by = nodes['DominatedBy']/7,
                                    fi_color = Blues8,
                                    fi_color2 = Greens8,
                                    age = nodes['age']/60,
                                    tgtdivsrcstgt=nodes['WonPCGT']*20,
                                    tgtdivsrc = nodes['Tgts/Srcs']*3
#dom_index is fill for circles
#wonpcgt is fill for rect
size="size_dim", fill_alpha="dom_index",line_color = "firebrick",
                                            line_dash = [1, 1],
                                            line_width = "dom_by",
                                            fill_color="fi_color"
     
     Rect(x="age", y="age",width="age",height="age",angle=-0.7,
                                            fill_alpha="tgtdivsrcstgt",
                                            line_color = "#f46d43",
                                            line_dash = "solid",
                                            line_width = "tgtdivsrc",
                                            fill_color="fi_color2"

In [113]:
#Create Edge Attributes:
color_series2 = []
for i in nodes['WonPCGT']:
    if i<0.2:
        color_series2.append(Oranges8[0])
    elif i<0.3:
        color_series2.append(Oranges8[1])
    elif i<0.4:
        color_series2.append(Oranges8[2])
    elif i<0.5:
        color_series2.append(Oranges8[3])
    elif i<0.6:
        color_series2.append(Oranges8[4])
    elif i<0.7:
        color_series2.append(Oranges8[5])
    elif i<0.8:
        color_series2.append(Oranges8[6])
    else:
        color_series2.append(Oranges8[7])

In [114]:
nodes['WonColor'] = color_series2

In [101]:
color_series3 = []
for i in 1/nodes['DomBy/DomOthers']:
    if i<0.5:
        color_series3.append(Blues8[0])
    elif i<1:
        color_series3.append(Blues8[1])
    elif i<1.5:
        color_series3.append(Blues8[2])
    elif i<2:
        color_series3.append(Blues8[3])
    elif i<2.5:
        color_series3.append(Blues8[4])
    elif i<3:
        color_series3.append(Blues8[5])
    elif i<3.5:
        color_series3.append(Blues8[6])
    else:
        color_series3.append(Blues8[7])

In [102]:
nodes['DomBy/DomOthers_color'] = color_series3

In [133]:
color_series4 = []
for i in nodes['Tgts/Srcs']:
    if i<1:
        color_series4.append(Greens8[0])
    elif i<2:
        color_series4.append(Greens8[1])
    elif i<3:
        color_series4.append(Greens8[2])
    elif i<4:
        color_series4.append(Greens8[3])
    elif i<6:
        color_series4.append(Greens8[4])
    elif i<8:
        color_series4.append(Greens8[5])
    elif i<9:
        color_series4.append(Greens8[6])
    else:
        color_series4.append(Greens8[7])

In [134]:
nodes['Tgts/Srcs_color'] = color_series4

In [132]:
nodes['Tgts/Srcs']

0     6.000000
1     2.500000
2     2.111111
3     1.800000
4     0.500000
5     0.050000
6     8.500000
7     3.000000
8     2.428571
9     2.000000
10    1.571429
11    1.600000
12    1.875000
13    1.000000
14    1.200000
15    0.909091
16    0.727273
17    0.785714
18    0.571429
19    0.166667
20    0.090909
21    4.250000
22    0.769231
23    0.583333
24    0.538462
25    0.200000
26    0.166667
27    4.000000
Name: Tgts/Srcs, dtype: float64

In [109]:
nodes['Tgts/Srcs_color']

0     #f7fcf5
1     #c7e9c0
2     #41ab5d
3     #41ab5d
4     #005a32
5     #005a32
6     #f7fcf5
7     #a1d99b
8     #a1d99b
9     #74c476
10    #41ab5d
11    #a1d99b
12    #41ab5d
13    #238b45
14    #238b45
15    #238b45
16    #005a32
17    #41ab5d
18    #005a32
19    #005a32
20    #005a32
21    #f7fcf5
22    #005a32
23    #238b45
24    #41ab5d
25    #005a32
26    #005a32
27    #a1d99b
Name: Tgts/Srcs_color, dtype: object

In [131]:
nodes

,id,age,DominatedBy,DominatedOthers,TargetsPerSource,SourcesPerTarget,DomBy/DomOthers,Tgts/Srcs,DominatedOthersPCGT,DominatedByPCGT,TotalInteractions,WonPCGT,LostPCGT,WonColor,DomBy/DomOthers_color,Tgts/Srcs_color
0,1,8,1,6,6,1,0.266667,6.000000,0.008746,0.001458,7,0.857143,0.142857,#fff5eb,#f7fbff,#005a32
1,2,8,10,36,15,6,0.377778,2.500000,0.052478,0.014577,46,0.782609,0.217391,#fee6ce,#c6dbef,#005a32
2,3,4,37,64,19,9,0.678125,2.111111,0.093294,0.053936,101,0.633663,0.366337,#fdd0a2,#4292c6,#005a32
3,4,7,16,26,18,10,0.715385,1.800000,0.037901,0.023324,42,0.619048,0.380952,#fdd0a2,#4292c6,#005a32
4,5,8,6,3,3,6,2.100000,0.500000,0.004373,0.008746,9,0.333333,0.666667,#f16913,#084594,#41ab5d
5,6,3,63,1,1,20,63.100000,0.050000,0.001458,0.091837,64,0.015625,0.984375,#8c2d04,#084594,#f7fcf5
6,7,5,2,44,17,2,0.145455,8.500000,0.064140,0.002915,46,0.956522,0.043478,#fff5eb,#f7fbff,#005a32
7,8,4,19,57,15,5,0.433333,3.000000,0.083090,0.027697,76,0.750000,0.250000,#fee6ce,#9ecae1,#005a32
8,9,7,19,53,17,7,0.458491,2.428571,0.077259,0.027697,72,0.736111,0.263889,#fee6ce,#9ecae1,#005a32
9,10,9,2,4,4,2,0.600000,2.000000,0.005831,0.002915,6,0.666667,0.333333,#fdd0a2,#6baed6,#005a32
